<a href="https://colab.research.google.com/github/ilank-pro/Goals/blob/main/Falcon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip show yfinance
! pip install curl_cffi



Name: yfinance
Version: 0.2.57
Summary: Download market data from Yahoo! Finance API
Home-page: https://github.com/ranaroussi/yfinance
Author: Ran Aroussi
Author-email: ran@aroussi.com
License: Apache
Location: /usr/local/lib/python3.11/dist-packages
Requires: beautifulsoup4, frozendict, multitasking, numpy, pandas, peewee, platformdirs, pytz, requests
Required-by: 


In [ ]:
#! pip uninstall yfinance
#! pip install --upgrade pip
# !pip install --upgrade yfinance==0.2.54
# !pip install yfinance --upgrade --no-cache-dir


In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import webbrowser
from IPython.display import HTML


In [ ]:
  from curl_cffi import requests
  session = requests.Session(impersonate="chrome")
  ticker = yf.Ticker('...', session=session)

In [ ]:
# Step 1: Retrieve the list of Nasdaq-100 stocks
nasdaq_100_tickers = """
AAPL MSFT AMZN NVDA GOOG GOOGL META TSLA PEP ADBE NFLX CSCO AVGO TXN CMCSA COST INTU SBUX AMGN QCOM AMD BKNG GILD ADP MU MDLZ LRCX ADI HON LLY AMAT ISRG PYPL NOW REGN PANW ORLY TMUS CHTR ABNB ILMN ZM SNPS MRVL FTNT KLAC DXCM NXPI VRTX MS DELL BIIB WDAY EA EXC SEDG MCHP PDD ASML IDXX KDP MAR WMT CDNS ALGN MRNA OKTA PTON DOCU TEAM ZS MDB DDOG SHOP CRSP NET U DKNG SQ PATH BILL ROKU TWLO RIVN CRWD
"""

# Step 2: Retrieve the S&P 500 tickers (using yfinance)
def get_sp500_tickers():
    sp500 = yf.Ticker("^GSPC").constituents
    return sp500["Symbol"].tolist()

try:
    sp500_tickers = get_sp500_tickers()
except Exception as e:
    print("Failed to fetch S&P 500 tickers. Falling back to a static list.")
    sp500_tickers = """
    MMM AOS ABT ABBV ACN ADM ADBE ADP AAP AES AFL AIG APD AKAM ALB ALGN ALL AMZN AMCR AEE AAL AEP AXP AIG ALK ALLY AMGN AME AMT AMP ANET AON APA AAPL AMAT APH ADI ANSS AON APA APTV ADM AIZ T ADP ADSK ADT AVB AVY BKR BAX BDX BAC BK BAX BWA BBWI BDX BAC BAX BBY BIIB BIO BA BKNG BWA BSX BXP CACI CARR CCL CCL CAT CCI CCI CF CE CHTR CINF CTSH CLX CMS CL CMC COST CTRA COP CMI CAG CSCO CSX CMCSA CMS CVX CINF CB CHD CCL CFG CHRW CLF CL COF CPRT CCOI
    """.split()

#https://docs.google.com/spreadsheets/d/1nyRsUk4a5FsESr_lGAfiB-Hg674lvCkI/edit?gid=1430016460#gid=1430016460
energy_tickers = """
SMR NNE CCJ BEP CEG D UEC OKLO LEU VST TLN NEE LTBR BWXT GE HTHIY
"""

# Combine Nasdaq-100 and S&P 500 lists into one
tickers = list(set(nasdaq_100_tickers.split() + sp500_tickers + energy_tickers.split() ))

Failed to fetch S&P 500 tickers. Falling back to a static list.


In [ ]:
# import pandas as pd

# # Generate a template CSV file for alerts
# def generate_alerts_csv(filename="alerts.csv"):
#     data = {
#         "Stock": ["AAPL", "TSLA", "AMZN"],  # Example stocks
#         "Trigger Price": [150, 700, 3300],
#         "Direction": ["above", "below", "between"],
#         "Date Duration": ["11/01/2023 until 11/30/2023", "11/01/2023 until 11/20/2023", "11/15/2023 until 11/30/2023"],
#         "Alert Priority": ["critical", "major", "minor"],
#         "Alert Alias": ["Breakout Alert", "Pullback Alert", "Range Alert"]
#     }
#     df = pd.DataFrame(data)
#     df.to_csv(filename, index=False)
#     print(f"Alerts CSV file generated: {filename}")

# generate_alerts_csv()

In [ ]:
# Helper function to create Yahoo Finance links
def create_yahoo_link(ticker):
    #return f'<a href="https://finance.yahoo.com/quote/{ticker}" target="_blank">{ticker}</a>'
    return f'<a href="https://finviz.com/quote.ashx?t={ticker}&p=d" target="_blank">{ticker}</a>'


# Function to fetch stock data
def get_stock_data(tickers, period="1y", interval="1d"):
    data = {}
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(period=period, interval=interval)
            data[ticker] = hist

        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
    return data



In [ ]:
def filter_stocks_with_earnings(data, alerts_csv="alerts.csv"):
    # Load alerts from CSV
    alerts = pd.read_csv(alerts_csv)

    results = {
        "Alerted Stocks": [],
        "Small Stocks": [],
        "Medium Stocks": [],
        "Prime Stocks": [],
        "Surging Stocks": [],
        "Hot Zone Stocks": [],
        "Earnings in Next 7 Days": [],
        "Earnings in 7-14 Days": []
    }

    for ticker, df in data.items():
        try:
            stock = yf.Ticker(ticker)
            earnings_date = None

            try:
                calendar = stock.calendar

                if 'Earnings Date' in calendar:
                    earnings_date = calendar['Earnings Date'][0]
                else:
                    earnings_date = None

            except Exception as e:
                print(f"An error occurred while getting Earnings Date: {e}")

            # Calculate days to earnings
            days_to_earnings = None
            if earnings_date is not None:
                earnings_date = pd.Timestamp(earnings_date)
                days_to_earnings = (earnings_date - pd.Timestamp.today()).days

            # Add stock metrics
            df['7d_change'] = (df['Close'] - df['Close'].shift(7)) / df['Close'].shift(7) * 100
            df['3d_change'] = (df['Close'] - df['Close'].shift(3)) / df['Close'].shift(3) * 100
            df['1d_change'] = (df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1) * 100
            df['Volume Change %'] = (df['Volume'] - df['Volume'].rolling(20).mean()) / df['Volume'].rolling(20).mean() * 100
            df['8d_MA'] = df['Close'].rolling(8).mean()
            df['20d_MA'] = df['Close'].rolling(20).mean()
            df['50d_MA'] = df['Close'].rolling(50).mean()

            latest = df.iloc[-1]
            price = latest['Close']
            volume = latest['Volume']
            avg_volume = df['Volume'].rolling(20).mean().iloc[-1]
            volume_change = latest['Volume Change %']

            # Call breaking pattern function
            breakout_pattern = identify_breaking_patterns(df, price, volume)

             # Check for active alerts
            alert_alias = check_alerts_for_stock(ticker, price, alerts)



            # Classify stocks into groups
            stock_entry = (ticker, price, latest['1d_change'], latest['3d_change'], latest['7d_change'], volume, volume_change, days_to_earnings, breakout_pattern)

            if alert_alias:
                results['Alerted Stocks'].append(stock_entry)
            if breakout_pattern != "No Breakout":
                if 2 <= price <= 10:
                    if 4.75 <= price <= 5.25 or 9.5 <= price <= 10.5:
                        results['Small Stocks'].append(stock_entry)
                elif 10 < price <= 50:
                    results['Medium Stocks'].append(stock_entry)
                elif price > 50 and volume > 4e6:
                    results['Prime Stocks'].append(stock_entry)

            # Surging Stocks: Volume >= 1.5x of 20-day average volume
            if volume >= 1.5 * avg_volume:
                results['Surging Stocks'].append(stock_entry)

            # Hot Zone Stocks: Volume >= 3x of 20-day average volume
            if volume >= 3 * avg_volume:
                results['Hot Zone Stocks'].append(stock_entry)

            # Filter for earnings dates
            if days_to_earnings is not None:
                if 0 <= days_to_earnings <= 7:
                    results['Earnings in Next 7 Days'].append(stock_entry)
                elif 7 < days_to_earnings <= 14:
                    results['Earnings in 7-14 Days'].append(stock_entry)

        except Exception as e:
            print(f"Error processing {ticker}: {e}")

    return results

In [ ]:
# Update display_tables to include Breakout Pattern and Days to Earnings columns
def display_tables_with_earnings(results):
    # Include DataTables integration as before
    display(HTML("""
    <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.13.6/css/jquery.dataTables.min.css">
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    <script src="https://cdn.datatables.net/1.13.6/js/jquery.dataTables.min.js"></script>
    <script>
        $(document).ready(function() {
            $('table.dataframe').DataTable({
                paging: false,
                searching: false
            });
        });
    </script>
    """))

    for group, stocks in results.items():
        if stocks:
            # Update the DataFrame to include the Breakout Pattern column
            df = pd.DataFrame(stocks, columns=[
                "Ticker", "Price", "1D Change (%)", "3D Change (%)",
                "7D Change (%)", "Volume", "Volume Change (%)",
                "Days to Earnings" , "Breakout Pattern"
            ])
            df['Ticker'] = df['Ticker'].apply(lambda x: create_yahoo_link(x))

            # Format numbers
            df['Price'] = df['Price'].map(lambda x: f"{x:.2f}")
            df['1D Change (%)'] = df['1D Change (%)'].map(lambda x: f"{x:.2f}%")
            df['3D Change (%)'] = df['3D Change (%)'].map(lambda x: f"{x:.2f}%")
            df['7D Change (%)'] = df['7D Change (%)'].map(lambda x: f"{x:.2f}%")
            df['Volume'] = df['Volume'].map(lambda x: f"{x / 1e6:.2f}M")
            df['Volume Change (%)'] = df['Volume Change (%)'].map(lambda x: f"{x:.2f}%")

            df['Breakout Pattern'] = df['Breakout Pattern'].map(lambda x: x if x else "No Pattern")
            df['Days to Earnings'] = df['Days to Earnings'].map(lambda x: f"{x} days" if x is not None and x >= 0 else "N/A")

            display(HTML(f"<h3>{group}</h3>"))
            display(HTML(df.to_html(escape=False, index=False, classes="dataframe")))

In [ ]:


# Export results to tables, including Breakout Pattern
def export_to_excel(results, filename="stock_report.xlsx"):
    with pd.ExcelWriter(filename) as writer:
        for group, stocks in results.items():
            # Include Breakout Pattern column
            df = pd.DataFrame(stocks, columns=[
                "Ticker", "Price", "1D Change (%)", "3D Change (%)",
                "7D Change (%)", "Volume", "Volume Change (%)",
                "Days to Earnings", "Breakout Pattern"
            ])
            df['Ticker'] = df['Ticker'].apply(lambda x: f'=HYPERLINK("{create_yahoo_link(x)}", "{x}")')
            df.to_excel(writer, sheet_name=group, index=False)
    print(f"Exported report to {filename}")


In [ ]:

ticker1 = "AAPL"

stock = yf.Ticker(ticker1)

# Fetch the calendar data, which includes upcoming events like the earnings date
try:
    calendar = stock.calendar

    # Check if 'Earnings Date' is available in the calendar data
    if 'Earnings Date' in calendar:
        earnings_date = calendar['Earnings Date'][0]  # Take the first date if available
        print(f"The next earnings date for {ticker1} is: {earnings_date}")
    else:
        print("Next earnings date not available in the calendar data.")

except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: 'str' object has no attribute 'name'


In [ ]:
# Plot stock performance
#def plot_stock_performance(data, tickers, period_ranges=[3, 5, 7, 14]):
def plot_stock_performance(data, tickers, period_ranges=[3, 14]):
    for ticker in tickers:
        if ticker not in data:
            continue
        df = data[ticker]
        plt.figure(figsize=(10, 6))
        for period in period_ranges:
            df[f'{period}d_change'] = (df['Close'] - df['Close'].shift(period)) / df['Close'].shift(period) * 100
            plt.plot(df.index, df[f'{period}d_change'], label=f'{period}-day % Change')
        plt.title(f"Performance for {ticker}")
        plt.xlabel("Date")
        plt.ylabel("% Change")
        plt.legend()
        plt.show()

In [ ]:
import numpy as np
import pandas as pd

def identify_breaking_patterns(df, price, volume):
    """
    Identify breaking patterns for a stock based on price, volume, and other metrics.
    """
    breakout_patterns = []

    # Ensure the DataFrame has enough rows for moving average calculations
    if len(df) < 50:
        return "Insufficient Data for Breakout Analysis"

    # Ensure indicators are calculated
    #df = calculate_indicators(df)

    # Calculate resistance and support levels
    resistance_level = df['Close'].max() * 1.02  # Ceiling: 2% above the maximum close
    support_level = df['Close'].min() * 0.98    # Floor: 2% below the minimum close

    # 1. Fibonacci Retracement Breakout
    high = df['Close'].max()
    low = df['Close'].min()
    fib_levels = {
        "61.8%": high - (0.618 * (high - low)),
        "50%": high - (0.5 * (high - low)),
        "38.2%": high - (0.382 * (high - low))
    }
    if any(abs(price - level) < 0.5 for level in fib_levels.values()):
        breakout_patterns.append("Fibonacci Retracement Breakout")

    # 2. Bollinger Band Breakout
    if 'Bollinger_Upper' in df.columns and not df['Bollinger_Upper'].isnull().all():
        if price > df['Bollinger_Upper'].iloc[-1]:  # Upper breakout
            breakout_patterns.append("Bollinger Band Breakout")

    # 4. Trendline Breakout (Simple example with linear regression)
    if len(df) > 20:
        x = np.arange(len(df))
        coef = np.polyfit(x, df['Close'], 1)
        trendline = np.polyval(coef, x)
        if price > trendline[-1]:  # Breakout above trendline
            breakout_patterns.append("Trendline Breakout")
    # 5: Gateway Selection Breakout
    gateway_prices = [10, 20, 50, 100, 200, 300]
    if any(abs(price - gateway) <= gateway * 0.05 for gateway in gateway_prices):
        breakout_patterns.append("Gateway Selection")

    # 6. Cup and Handle Breakout
    # if len(df) > 30:
    #     # Find the index of the minimum value in the last 30 days
    #     min_idx = df['Close'].iloc[-30:].idxmin()

    #     # Calculate the start of the cup by subtracting 15 days
    #     cup_start_idx = max(min_idx - 15, 0)

    #     # Ensure valid bounds for slicing
    #     if min_idx > cup_start_idx:
    #         cup_max_price = df['Close'].iloc[cup_start_idx:min_idx].max()
    #         if price > cup_max_price:
    #             breakout_patterns.append("Cup and Handle Breakout")

    # 7. Head and Shoulders Breakout
    if len(df) > 30:
        shoulders = df['Close'].iloc[-30:-15]
        head = df['Close'].iloc[-15:]
        if head.max() > shoulders.max() and price < shoulders.min():
            breakout_patterns.append("Head and Shoulders Breakout")

    # 9. Gap Up/Down Breakout
    if len(df) > 1:
        gap = abs(df['Close'].iloc[-2] - df['Open'].iloc[-1])
        if gap > (df['Close'].std() * 2):
            breakout_patterns.append("Gap Breakout")

    # 11. Symmetrical Triangle Breakout
    if len(df) > 20:
        recent_highs = df['High'].iloc[-20:].rolling(5).max()
        recent_lows = df['Low'].iloc[-20:].rolling(5).min()
        if recent_highs.is_monotonic_decreasing and recent_lows.is_monotonic_increasing and price > recent_highs.iloc[-1]:
            breakout_patterns.append("Symmetrical Triangle Breakout")

    # 13. Pivot Point Breakout
    pivot = (high + low + df['Close'].iloc[-1]) / 3
    resistance1 = (2 * pivot) - low
    support1 = (2 * pivot) - high
    if abs(price - resistance1) < 0.5:
        breakout_patterns.append("Pivot Resistance Breakout")
    if abs(price - support1) < 0.5:
        breakout_patterns.append("Pivot Support Breakout")

    # 14. Triple Top/Bottom Breakout
    if len(df) > 30:
        recent_close = df['Close'].iloc[-30:]
        top = recent_close.max()
        bottom = recent_close.min()
        if recent_close.iloc[-1] > top * 0.98:  # Breakout above triple top
            breakout_patterns.append("Triple Top Breakout")
        if recent_close.iloc[-1] < bottom * 1.02:  # Breakdown below triple bottom
            breakout_patterns.append("Triple Bottom Breakout")

    # 15. Keltner Channel Breakout
    if 'Keltner_Upper' in df.columns and not df['Keltner_Upper'].isnull().all():
        if price > df['Keltner_Upper'].iloc[-1]:
            breakout_patterns.append("Keltner Channel Breakout")

    # Breakout conditions
    if price > df['Close'].max():  # New 52-week high breakout
        breakout_patterns.append("52-Week High Breakout")
    if '50d_MA' in df.columns:
        # Check if the crossing occurred in the last 3 days
        recent_cross = any(
            abs(df['Close'].iloc[-i] - df['50d_MA'].iloc[-i]) < 1e-6  # Close price approximately equal to 50d MA
            for i in range(1, 4)  # Check the last 3 days
        )
        if recent_cross:
            breakout_patterns.append("50-Day MA Touch")
    if '20d_MA' in df.columns:
        # Check if the crossing occurred in the last 3 days
        recent_cross = any(
            abs(df['Close'].iloc[-i] - df['20d_MA'].iloc[-i]) < 1e-6  # Close price approximately equal to 20d MA
            for i in range(1, 4)  # Check the last 3 days
        )
        if recent_cross:
            breakout_patterns.append("20-Day MA Touch")
    # if 'Volume' in df.columns and volume >= 1.5 * df['Volume'].rolling(20).mean().iloc[-1]:  # Volume spike breakout
    #     breakout_patterns.append("Volume Spike Breakout")
    if '8d_MA' in df.columns and '50d_MA' in df.columns:
        # Check if the crossover occurred in the last 3 days using explicit iloc
        recent_cross = (df['8d_MA'].iloc[-3:] > df['50d_MA'].iloc[-3:]).any() and (df['8d_MA'].iloc[-4] <= df['50d_MA'].iloc[-4])
        if recent_cross:
            breakout_patterns.append("8-Day Crossing")
    if abs(price - resistance_level) <= 0.5:  # Ceiling resistance
        breakout_patterns.append("Ceiling Resistance")
    if abs(price - support_level) <= 0.5:  # Floor resistance
        breakout_patterns.append("Floor Resistance")

    # Finviz-style patterns
    # if 'High' in df.columns and 'Low' in df.columns:
    #     if df['High'].iloc[-1] > df['High'].rolling(3).mean().iloc[-1]:  # Ascending triangle
    #         breakout_patterns.append("Ascending Triangle")
    #     if df['Low'].iloc[-1] < df['Low'].rolling(3).mean().iloc[-1]:  # Descending triangle
    #         breakout_patterns.append("Descending Triangle")

    # Additional patterns
    if 'RSI' in df.columns and df['RSI'].iloc[-1] > 80:  # RSI overbought breakout
        breakout_patterns.append("RSI Overbought")
    if 'RSI' in df.columns and df['RSI'].iloc[-1] < 25:  # RSI oversold breakout
        breakout_patterns.append("RSI Oversold")
    if 'Bollinger_Upper' in df.columns and price > df['Bollinger_Upper'].iloc[-1]:  # Bollinger band breakout
        breakout_patterns.append("Bollinger Band Breakout")
    if 'Bollinger_Lower' in df.columns and price < df['Bollinger_Lower'].iloc[-1]:  # Bollinger band breakdown
        breakout_patterns.append("Bollinger Band Breakdown")


    return " + ".join(breakout_patterns) if breakout_patterns else "No Breakout"

In [ ]:
from datetime import datetime

def check_alerts_for_stock(ticker, price, alerts):
    """
    Check if the given stock meets any alert conditions.
    """
    active_alerts = []
    for _, alert in alerts.iterrows():
        if alert["Stock"] == ticker:
            trigger_price = alert["Trigger Price"]
            direction = alert["Direction"]
            date_duration = alert["Date Duration"]
            priority = alert["Alert Priority"]
            alias = alert["Alert Alias"]

            # Parse date duration
            try:
                start_date, end_date = date_duration.split(" until ")
                start_date = datetime.strptime(start_date.strip(), "%m/%d/%Y")
                end_date = datetime.strptime(end_date.strip(), "%m/%d/%Y")
                today = datetime.today()

                # Check if within date duration
                if start_date <= today <= end_date:
                    # Handle different directions
                    if direction == "above":
                        if price > float(trigger_price):
                            active_alerts.append((priority, alias))
                    elif direction == "below":
                        if price < float(trigger_price):
                            active_alerts.append((priority, alias))
                    elif direction == "between":
                        # Parse range from trigger_price
                        try:
                            lower, upper = map(float, trigger_price.split('-'))
                            if lower <= price <= upper:
                                active_alerts.append((priority, alias))
                        except ValueError as e:
                            print(f"Error parsing 'between' range for {ticker}: {e}")
            except Exception as e:
                print(f"Error parsing alert for {ticker}: {e}")
                continue

    # Return the highest-priority alert if multiple are active
    if active_alerts:
        active_alerts.sort(key=lambda x: {"critical": 0, "major": 1, "minor": 2}[x[0]])
        return active_alerts[0][1]  # Return alias of highest-priority alert
    return ""

In [ ]:
def display_tables_with_alerts(results, alerts_csv="alerts.csv"):
    # Load alerts CSV
    alerts = pd.read_csv(alerts_csv)

    # Include DataTables integration with multi-select filter for "Breakout Pattern"
    display(HTML("""
    <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.13.6/css/jquery.dataTables.min.css">
    <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/buttons/2.3.6/css/buttons.dataTables.min.css">
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    <script src="https://cdn.datatables.net/1.13.6/js/jquery.dataTables.min.js"></script>
    <script src="https://cdn.datatables.net/buttons/2.3.6/js/dataTables.buttons.min.js"></script>
    <script src="https://cdn.datatables.net/select/1.5.0/js/dataTables.select.min.js"></script>
    <script>
        $(document).ready(function() {
            $('table.dataframe').DataTable({
                paging: false,
                searching: true,
                dom: 'Bfrtip',
                buttons: [],
                initComplete: function () {
                    var column = this.api().columns(7);  // "Breakout Pattern" is column 7 (index starts at 0)
                    var select = $('<select multiple="multiple" style="width: 100%;"><option value=""></option></select>')
                        .appendTo($(column.header()))
                        .on('change', function () {
                            var val = $(this).val();
                            column
                                .search(val.join('|'), true, false)  // Regex search for multi-selection
                                .draw();
                        });

                    column.data().unique().sort().each(function (d, j) {
                        select.append('<option value="' + d + '">' + d + '</option>')
                    });
                }
            });
        });
    </script>
    """))

    active_alerts_data = []

    for group, stocks in results.items():
        if stocks:
            # Create DataFrame
            df = pd.DataFrame(stocks, columns=[
                "Ticker", "Price", "1D Change (%)", "3D Change (%)",
                "7D Change (%)", "Volume", "Volume Change (%)",
                "Days to Earnings", "Breakout Pattern"
            ])
            df['Alerts'] = df.apply(
                lambda row: check_alerts_for_stock(row['Ticker'], row['Price'], alerts), axis=1
            )

            # Format numbers
            df['Price'] = df['Price'].map(lambda x: f"{x:.2f}")
            df['1D Change (%)'] = df['1D Change (%)'].map(lambda x: f"{x:.2f}%")
            df['3D Change (%)'] = df['3D Change (%)'].map(lambda x: f"{x:.2f}%")
            df['7D Change (%)'] = df['7D Change (%)'].map(lambda x: f"{x:.2f}%")
            df['Volume'] = df['Volume'].map(lambda x: f"{x / 1e6:.2f}M")
            df['Volume Change (%)'] = df['Volume Change (%)'].map(lambda x: f"{x:.2f}%")
            df['Days to Earnings'] = df['Days to Earnings'].map(lambda x: f"{x} days" if x is not None and x >= 0 else "N/A")

            # Collect active alerts
            active_alerts_data.extend(df[df['Alerts'] != ""].to_dict("records"))

            # Add clickable links to tickers
            df['Ticker'] = df['Ticker'].apply(lambda x: create_yahoo_link(x))

            # Display the table for the current group
            display(HTML(f"<h3>{group}</h3>"))
            display(HTML(df.to_html(escape=False, index=False, classes="dataframe")))

In [ ]:
# Main execution
if __name__ == "__main__":
    tickers = ["AAPL", "TSLA", "AMZN", "MSFT", "GOOGL"]  # Add more tickers as needed
    stock_data = get_stock_data(tickers)
    filtered_results = filter_stocks_with_earnings(stock_data)

    #export_to_excel(filtered_results)
    # Display tables in Jupyter Notebook
    #display_tables_with_earnings(filtered_results)
    display_tables_with_alerts(filtered_results)

    # Plot leading stocks (example: top stocks from Surging Stocks group)
    #top_tickers = [stock[0] for stock in filtered_results["Surging Stocks"][:5]]
    #plot_stock_performance(stock_data, top_tickers)

ERROR:yfinance:Failed to get ticker 'AAPL' reason: 'str' object has no attribute 'name'
ERROR:yfinance:$AAPL: possibly delisted; no price data found  (period=1y)
ERROR:yfinance:Failed to get ticker 'TSLA' reason: 'str' object has no attribute 'name'
ERROR:yfinance:$TSLA: possibly delisted; no price data found  (period=1y)
ERROR:yfinance:Failed to get ticker 'AMZN' reason: 'str' object has no attribute 'name'
ERROR:yfinance:$AMZN: possibly delisted; no price data found  (period=1y)
ERROR:yfinance:Failed to get ticker 'MSFT' reason: 'str' object has no attribute 'name'
ERROR:yfinance:$MSFT: possibly delisted; no price data found  (period=1y)
ERROR:yfinance:Failed to get ticker 'GOOGL' reason: 'str' object has no attribute 'name'
ERROR:yfinance:$GOOGL: possibly delisted; no price data found  (period=1y)


An error occurred while getting Earnings Date: 'str' object has no attribute 'name'
Error processing AAPL: single positional indexer is out-of-bounds
An error occurred while getting Earnings Date: 'str' object has no attribute 'name'
Error processing TSLA: single positional indexer is out-of-bounds
An error occurred while getting Earnings Date: 'str' object has no attribute 'name'
Error processing AMZN: single positional indexer is out-of-bounds
An error occurred while getting Earnings Date: 'str' object has no attribute 'name'
Error processing MSFT: single positional indexer is out-of-bounds
An error occurred while getting Earnings Date: 'str' object has no attribute 'name'
Error processing GOOGL: single positional indexer is out-of-bounds
